# Basics of Trust Region Controllers in Xopt

Trust Region Bayesian Optimization (TuRBO) is an advanced optimization algorithm designed for solving high-dimensional black-box optimization problems. It combines the strengths of Bayesian Optimization (BO) with trust region methods to improve scalability and efficiency.

### Key Features:
1. **Trust Regions**:
    - TuRBO uses local trust regions to focus the search in promising areas of the parameter space.
    - Each trust region is a bounded subspace where the optimization is performed, and its size is dynamically adjusted based on the success of the optimization.

2. **Bayesian Surrogate Model**:
    - A Gaussian Process (GP) or other surrogate models are used to approximate the objective function.
    - This surrogate model is used to predict the objective function and guide the search as well as define the size of the trust region.

4. **Adaptivity**:
    - The algorithm adapts the size of the trust region based on the success or failure of the optimization steps. If the optimization within a trust region is successful, the region expands; otherwise, it shrinks.

### Advantages:
- Scales better to high-dimensional problems compared to standard Bayesian Optimization.
- Efficiently balances exploration and exploitation within trust regions.

### Disadvantages:
- Severely restricts exploration of the parameter space, potentially leading to convergence to local minima, thus making it sensative to initial sampling points.
- Introduces additional algorithm hyperparameters which can cause issues.  
- May struggle with highly noisy objective functions or discontinuous landscapes.  

## Defining a TuRBO Controller
Currently, Xopt supports 3 different TuRBO controller types, the most basic of which is the `OptimizeTurboController`. To create this controller we need to define our optimization problem and some data.

In [1]:
import numpy as np
from xopt import VOCS
import pandas as pd

# create evaluation function
def sphere_function(inputs):
    """
    2D Sphere objective function.
    Compatible with Xopt.
    """
    x, y = inputs["x"], inputs["y"]
    return {"f": np.sum(np.square(np.stack([x, y], axis=-1)), axis=-1)}

# create a VOCS object
vocs = VOCS(
    variables = {
        "x": {-5,5},
        "y": {-5,5}
    },
    objectives = {"f": "MINIMIZE"},
)

# random sample 10 points
x0 = vocs.random_inputs(10)

# evaluate the function at the random points
f = []
for i in range(len(x0)):
    f += [sphere_function(x0[i]) | x0[i]]

# print the results
data = pd.DataFrame(f)
data

,f,x,y
0,28.568032,-2.846720,4.523740
1,11.431360,2.469166,-2.309671
2,35.055441,-3.313917,-4.906465
3,20.463050,2.296225,-3.897486
4,35.433982,-4.624704,3.747812
5,6.386172,2.280505,1.088792
6,23.422974,-4.506553,-1.764639
7,2.116901,1.353808,0.533015
8,12.713870,-2.905213,-2.067271
9,21.249261,-4.427933,-1.281669


## Create the ExpectedImprovementGenerator and train the GP model
Here we create the ExpectedImprovementGenerator, add data to the generator, and train the model from the data.

In [2]:
from xopt.generators.bayesian import ExpectedImprovementGenerator
generator = ExpectedImprovementGenerator(vocs=vocs) # create the generator
generator.add_data(data) # add the data to the generator
generator.train_model() # train the model

ModelListGP(
  (models): ModuleList(
    (0): SingleTaskGP(
      (likelihood): GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
      (mean_module): ConstantMean()
      (covar_module): RBFKernel(
        (lengthscale_prior): LogNormalPrior()
        (raw_lengthscale_constraint): GreaterThan(2.500E-02)
      )
      (outcome_transform): Standardize()
      (input_transform): Normalize()
    )
  )
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0): GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
)

## Create the Optimize Turbo Controller
Here we create the controller and view the different parameters with their descriptions.

In [3]:
from xopt.generators.bayesian.turbo import OptimizeTurboController
turbo_controller = OptimizeTurboController(vocs=vocs)

print(turbo_controller.__doc__)
print("-" * 20)

# examine the attributes of the controller
for field_name, field in turbo_controller.model_fields.items():
    print(f"Field: {field_name}")
    print(f"  Description: {field.description}")
    print(f"  Type: {field.annotation}")
    print(f"  Default: {field.default}")
    print(f"  Value: {getattr(turbo_controller, field_name)}")
    print("-" * 20)


    Turbo controller for optimization tasks.

    Attributes:
    -----------
    name : str
        The name of the controller.
    best_value : Optional[float]
        The best value found so far.

    Methods:
    --------
    vocs_validation(cls, info)
        Validate the VOCS for the controller.
    minimize(self) -> bool
        Check if the objective is to minimize.
    _set_best_point_value(self, data)
        Set the best point value based on the data.
    update_state(self, generator, previous_batch_size: int = 1) -> None
        Update the state of the controller.
    
--------------------
Field: vocs
  Description: VOCS object
  Type: <class 'xopt.vocs.VOCS'>
  Default: PydanticUndefined
  Value: variables={'x': [-5.0, 5.0], 'y': [-5.0, 5.0]} constraints={} objectives={'f': 'MINIMIZE'} constants={} observables=[]
--------------------
Field: dim
  Description: number of dimensions in the optimization problem
  Type: <class 'int'>
  Default: None
  Value: 2
----------------

## Getting the Trust Region
Here we get the trust region 


In [5]:
trust_region = turbo_controller.get_trust_region(generator=generator) # get the trust region of the model
print(f"Trust Region: {trust_region}")

Trust Region: tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
